In [1]:
import random
import pickle

import numpy as np
import pandas as pd
from nltk.tokenize import RegexpTokenizer


from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, Dense, Activation
from tensorflow.keras.optimizers import RMSprop


In [2]:
text_df = pd.read_csv("fake_or_real_news.csv")

In [3]:
text_df

,id,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL
...,...,...,...,...
6330,4490,State Department says it can't find emails fro...,The State Department told the Republican Natio...,REAL
6331,8062,The ‘P’ in PBS Should Stand for ‘Plutocratic’ ...,The ‘P’ in PBS Should Stand for ‘Plutocratic’ ...,FAKE
6332,8622,Anti-Trump Protesters Are Tools of the Oligarc...,Anti-Trump Protesters Are Tools of the Oligar...,FAKE
6333,4021,"In Ethiopia, Obama seeks progress on peace, se...","ADDIS ABABA, Ethiopia —President Obama convene...",REAL


In [4]:
text = list(text_df.text.values)
joined_text = " ".join(text)

In [5]:
partial_text = joined_text[:10000]

In [6]:
from nltk.tokenize import RegexpTokenizer

# Create a tokenizer with the desired regex pattern
tokenizer = RegexpTokenizer(r"\w+")


# Tokenize the text
tokens = tokenizer.tokenize(partial_text.lower())

# Print the tokens
print(tokens)


['daniel', 'greenfield', 'a', 'shillman', 'journalism', 'fellow', 'at', 'the', 'freedom', 'center', 'is', 'a', 'new', 'york', 'writer', 'focusing', 'on', 'radical', 'islam', 'in', 'the', 'final', 'stretch', 'of', 'the', 'election', 'hillary', 'rodham', 'clinton', 'has', 'gone', 'to', 'war', 'with', 'the', 'fbi', 'the', 'word', 'unprecedented', 'has', 'been', 'thrown', 'around', 'so', 'often', 'this', 'election', 'that', 'it', 'ought', 'to', 'be', 'retired', 'but', 'it', 's', 'still', 'unprecedented', 'for', 'the', 'nominee', 'of', 'a', 'major', 'political', 'party', 'to', 'go', 'war', 'with', 'the', 'fbi', 'but', 'that', 's', 'exactly', 'what', 'hillary', 'and', 'her', 'people', 'have', 'done', 'coma', 'patients', 'just', 'waking', 'up', 'now', 'and', 'watching', 'an', 'hour', 'of', 'cnn', 'from', 'their', 'hospital', 'beds', 'would', 'assume', 'that', 'fbi', 'director', 'james', 'comey', 'is', 'hillary', 's', 'opponent', 'in', 'this', 'election', 'the', 'fbi', 'is', 'under', 'attack',

In [7]:
unique_tokens = np.unique(tokens)
unique_token_index = {token: idx for idx, token in enumerate(unique_tokens)}

In [8]:
n_words = 10
input_words = []
next_words = []

for i in range(len(tokens) - n_words):
    input_words.append(tokens[i:i + n_words])
    next_words.append(tokens[i + n_words])

In [9]:
input_words


[['daniel',
  'greenfield',
  'a',
  'shillman',
  'journalism',
  'fellow',
  'at',
  'the',
  'freedom',
  'center'],
 ['greenfield',
  'a',
  'shillman',
  'journalism',
  'fellow',
  'at',
  'the',
  'freedom',
  'center',
  'is'],
 ['a',
  'shillman',
  'journalism',
  'fellow',
  'at',
  'the',
  'freedom',
  'center',
  'is',
  'a'],
 ['shillman',
  'journalism',
  'fellow',
  'at',
  'the',
  'freedom',
  'center',
  'is',
  'a',
  'new'],
 ['journalism',
  'fellow',
  'at',
  'the',
  'freedom',
  'center',
  'is',
  'a',
  'new',
  'york'],
 ['fellow',
  'at',
  'the',
  'freedom',
  'center',
  'is',
  'a',
  'new',
  'york',
  'writer'],
 ['at',
  'the',
  'freedom',
  'center',
  'is',
  'a',
  'new',
  'york',
  'writer',
  'focusing'],
 ['the',
  'freedom',
  'center',
  'is',
  'a',
  'new',
  'york',
  'writer',
  'focusing',
  'on'],
 ['freedom',
  'center',
  'is',
  'a',
  'new',
  'york',
  'writer',
  'focusing',
  'on',
  'radical'],
 ['center',
  'is',
  'a',
  

In [10]:
X = np.zeros((len(input_words), n_words, len(unique_tokens)), dtype=bool)
y = np.zeros((len(next_words), len(unique_tokens)), dtype=bool)

In [11]:
X

array([[[False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        ...,
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False]],

       [[False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        ...,
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False]],

       [[False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        ...,
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, Fal

In [12]:
y

array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])

In [13]:
for i, words in enumerate(input_words):
    for j, word in enumerate(words):
        X[i, j, unique_token_index[word]] = 1
    y[i, unique_token_index[next_words[i]]] = 1

In [14]:
model = Sequential()
model.add(LSTM(128, input_shape=(n_words, len(unique_tokens)), return_sequences=True))
model.add(LSTM(128))
model.add(Dense(len(unique_tokens)))
model.add(Activation("softmax"))

C:\Users\91741\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [15]:
model.compile(loss="categorical_crossentropy", optimizer=RMSprop(learning_rate=0.01), metrics=["accuracy"])
model.fit(X, y, batch_size=128, epochs=30, shuffle=True)
              

Epoch 1/30
14/14 ━━━━━━━━━━━━━━━━━━━━ 7s 75ms/step - accuracy: 0.0432 - loss: 6.3205
Epoch 2/30
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 70ms/step - accuracy: 0.0638 - loss: 5.8001
Epoch 3/30
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 71ms/step - accuracy: 0.0597 - loss: 5.7429
Epoch 4/30
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 68ms/step - accuracy: 0.0584 - loss: 5.7454
Epoch 5/30
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 70ms/step - accuracy: 0.0637 - loss: 5.7120
Epoch 6/30
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 68ms/step - accuracy: 0.0668 - loss: 5.6926
Epoch 7/30
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 70ms/step - accuracy: 0.0518 - loss: 5.6589
Epoch 8/30
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 72ms/step - accuracy: 0.0627 - loss: 5.6294
Epoch 9/30
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 72ms/step - accuracy: 0.0659 - loss: 5.4869
Epoch 10/30
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 71ms/step - accuracy: 0.0851 - loss: 5.2978
Epoch 11/30
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 70ms/step - accuracy: 0.0819 - loss: 5.1709
Epoch 12/30
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 70ms/step - accuracy:

In [16]:
model.save("mymodel.h5")

In [17]:
model = load_model("mymodel.h5")

In [18]:
possible = predict_next_word("He will have to look into this thing and he", 5)

NameError: name 'predict_next_word' is not defined

In [ ]:
import numpy as np

def predict_next_word(input_text, n_best):
    input_text = input_text.lower()
    X = np.zeros((1, n_words, len(unique_tokens)))
    
    # Prepare input for prediction
    for i, word in enumerate(input_text.split()):
        if word in unique_token_index:  # Ensure the word exists in the index
            X[0, i, unique_token_index[word]] = 1
        else:
            print(f"Warning: Word '{word}' not found in token index")
    
    # Make predictions
    predictions = model.predict(X)[0]
    
    # Get top n_best predictions (sorted by probability)
    top_n_indices = np.argsort(predictions)[-n_best:][::-1]  # Sort in descending order
    
    # Get the corresponding tokens for the top predictions
    top_n_words = [unique_tokens[i] for i in top_n_indices]
    
    return top_n_words

# Example usage:
possible = predict_next_word("He will have to look into this thing and he", 5)
print(possible)


In [ ]:
def generate_text(input_text, text_length, creativity=3):
    word_sequence = input_text.split()
    current = 0
    for _ in range(text_length):
        sub_sequence = " ".join(tokenizer.tokenize(" ".join(word_sequence).lower())[current:current+n_words])
        try:
            choice = unique_tokens[random.choice(predict_next_word(sub_sequence, creativity))]
        except:
            choice = random.choice(unique_tokens)
        word_sequence.append(choice)
        current += 1
    return " ".join(word_sequence)

In [ ]:
generate_text("He will have to look into yhis thing and he",100, 5)